# merging of satellite and ground data

In [1]:
import pandas as pd
import glob
import os
import numpy as np
from geopy.distance import geodesic

In [2]:
prec = pd.read_parquet('data/fedearroz_prec_diaria_2024.parquet')


In [3]:
prec= prec.rename(columns={'lat':'latitud','lon':'longitud', 'prec':'precipitation'})
prec['fuente']='fedearroz'

In [4]:
prec.station.nunique()

42

# Loading of satellital data

In [5]:
def generate_dates(start_date, end_date, num_rows):
    date_range = pd.date_range(start=start_date, end=end_date, periods=num_rows)
    return date_range

In [6]:
file_pattern_chirps = "./chirps_data/chirps-precipitation_*.csv"
files_chirps = glob.glob(file_pattern_chirps)

file_pattern_agera5 = "./agera5_data/agera5-precipitation_*.csv"
files_agera5 = glob.glob(file_pattern_agera5)


files_combined = files_chirps + files_agera5

In [7]:
grouped_data = []

for file in files_combined:
    # Extraer la información del nombre del archivo
    basename = os.path.basename(file)
    parts = basename.replace('.csv', '').split('_')
    #latitud y longitud si viene en el archivo
    start_date = parts[3]
    end_date = parts[4]
    longitud=parts[1]
    latitud=parts[2]
    source=parts[0]
    
    df = pd.read_csv(file)
    num_rows = len(df)
    df = df.rename(columns={'precipitation': 'precipitation'})
    df = df.rename(columns={'Precipitation_Flux': 'precipitation'})
    df['date'] = generate_dates(start_date, end_date, num_rows)
    df['longitud']=float(longitud)
    df['latitud']=float(latitud)
    df['fuente']=source
    df=df[['longitud','latitud', 'date','precipitation','fuente']]
    grouped_data.append(df)



In [8]:
arroz_sat = pd.concat(grouped_data)
arroz_sat['precipitation'] = arroz_sat['precipitation'].fillna(0)
arroz_sat

,longitud,latitud,date,precipitation,fuente
0,-74.91457,4.72333,2020-01-01,0.00,chirps-precipitation
1,-74.91457,4.72333,2020-01-02,0.00,chirps-precipitation
2,-74.91457,4.72333,2020-01-03,0.00,chirps-precipitation
3,-74.91457,4.72333,2020-01-04,0.00,chirps-precipitation
4,-74.91457,4.72333,2020-01-05,0.00,chirps-precipitation
...,...,...,...,...,...
360,-74.76385,4.78134,2023-12-27,0.00,agera5-precipitation
361,-74.76385,4.78134,2023-12-28,0.22,agera5-precipitation
362,-74.76385,4.78134,2023-12-29,0.46,agera5-precipitation
363,-74.76385,4.78134,2023-12-30,1.04,agera5-precipitation


def distance_cal(row):
    # Coordenadas aproximadas (lat, lon)
    coords_aprox = (row['lat'], row['lon'])
    
    # Coordenadas reales (latitud, longitud)
    coords_reales = (row['latitud'], row['longitud'])
    
    # Calcular la distancia usando geodesic de geopy
    distancia = geodesic(coords_aprox, coords_reales).kilometers
    
    return distancia


df_unicos['distancia'] = df_unicos.apply(distance_cal, axis=1)

df_unicos.head()


In [9]:
estaciones= prec[['latitud', 'longitud', 'dpto', 'mun', 'station']].drop_duplicates(keep='first').reset_index(drop=True)

mask = estaciones['station'] == 'FEDEARROZ_TELLO_SAN_DIEGO'

# Modificar los valores de latitud y longitud para esa fila, formateando con 4 cifras significativas sin ceros adicionales
estaciones.loc[mask, 'latitud'] = estaciones.loc[mask, 'latitud'].apply(lambda x: format(3.08083, '.5f').rstrip('0').rstrip('.'))
estaciones.loc[mask, 'longitud'] = estaciones.loc[mask, 'longitud'].apply(lambda x: format(-75.19083, '.5f').rstrip('0').rstrip('.'))




mask = estaciones['station'] == 'FEDEARROZ_VALLEDUPAR_LA_ESPERANZA'

# Modificar los valores de latitud y longitud para esa fila, formateando con 4 cifras significativas sin ceros adicionales
estaciones.loc[mask, 'latitud'] = estaciones.loc[mask, 'latitud'].apply(lambda x: format(10.48792, '.5f').rstrip('0').rstrip('.'))
estaciones.loc[mask, 'longitud'] = estaciones.loc[mask, 'longitud'].apply(lambda x: format(-73.18527, '.5f').rstrip('0').rstrip('.'))



mask = estaciones['station'] == 'FEDEARROZ_GRANADA_LOS_NARANJOS'

# Modificar los valores de latitud y longitud para esa fila, formateando con 4 cifras significativas sin ceros adicionales
estaciones.loc[mask, 'latitud'] = estaciones.loc[mask, 'latitud'].apply(lambda x: format(3.30278, '.5f').rstrip('0').rstrip('.'))
estaciones.loc[mask, 'longitud'] = estaciones.loc[mask, 'longitud'].apply(lambda x: format(-73.56722, '.5f').rstrip('0').rstrip('.'))




mask = estaciones['station'] == 'FEDEARROZ_GUAMO_RANCHO_ALEGRE'

# Modificar los valores de latitud y longitud para esa fila, formateando con 4 cifras significativas sin ceros adicionales
estaciones.loc[mask, 'latitud'] = estaciones.loc[mask, 'latitud'].apply(lambda x: format(4.202420, '.5f').rstrip('0').rstrip('.'))
estaciones.loc[mask, 'longitud'] = estaciones.loc[mask, 'longitud'].apply(lambda x: format(-74.976070, '.5f').rstrip('0').rstrip('.'))


estaciones.latitud = estaciones.latitud.astype('float64')
estaciones.longitud = estaciones.longitud.astype('float64')
estaciones


/var/tmp/ipykernel_35842/609776960.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['3.08083']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  estaciones.loc[mask, 'latitud'] = estaciones.loc[mask, 'latitud'].apply(lambda x: format(3.08083, '.5f').rstrip('0').rstrip('.'))
/var/tmp/ipykernel_35842/609776960.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['-75.19083']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  estaciones.loc[mask, 'longitud'] = estaciones.loc[mask, 'longitud'].apply(lambda x: format(-75.19083, '.5f').rstrip('0').rstrip('.'))


,latitud,longitud,dpto,mun,station
0,4.393330,-72.988890,META,UPIA,BARRANCA_DE_UPIA
1,10.487920,-73.185270,CESAR,VALLEDUPAR,FEDEARROZ_VALLEDUPAR_LA_ESPERANZA
2,8.571670,-74.556010,BOLIVAR,ACHI,FEDEARROZ_ACHI_LA_MANO_DE_DIOS
3,4.977630,-72.616430,CASANARE,TAURAMENA,FEDEARROZ_AGUAZUL_EL_CUSIANA
4,5.139600,-72.501290,CASANARE,AGUAZUL,FEDEARROZ_AGUAZUL_EL_PALMAR
5,3.246090,-75.239780,HUILA,AIPE,FEDEARROZ_AIPE_PIEDRAPINTADA
6,4.567210,-74.957520,TOLIMA,ALVARADO,FEDEARROZ_ALVARADO_CALICANTO
7,4.781340,-74.763850,TOLIMA,AMBALEMA,FEDEARROZ_AMBALEMA_GAMBA
8,4.971950,-74.899740,TOLIMA,ARMERO,FEDEARROZ_ARMERO_EL_PUENTE
9,2.698140,-75.298150,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA


In [10]:
arroz_info_sat = pd.merge(arroz_sat,estaciones,
                            how='right', on=['longitud', 'latitud'])
arroz_info_sat.tail()

,longitud,latitud,date,precipitation,fuente,dpto,mun,station
357175,-73.479924,7.475428,2020-12-27,4.350000,agera5-precipitation,CESAR,AGUAZUL,FEDEARROZ_SAN_ALBERTO
357176,-73.479924,7.475428,2020-12-28,3.910000,agera5-precipitation,CESAR,AGUAZUL,FEDEARROZ_SAN_ALBERTO
357177,-73.479924,7.475428,2020-12-29,11.940000,agera5-precipitation,CESAR,AGUAZUL,FEDEARROZ_SAN_ALBERTO
357178,-73.479924,7.475428,2020-12-30,31.790001,agera5-precipitation,CESAR,AGUAZUL,FEDEARROZ_SAN_ALBERTO
357179,-73.479924,7.475428,2020-12-31,16.590000,agera5-precipitation,CESAR,AGUAZUL,FEDEARROZ_SAN_ALBERTO


In [11]:
arroz_obs_sat = pd.concat([prec, arroz_info_sat], ignore_index=True)

In [12]:
arroz_obs_sat =arroz_obs_sat[(arroz_obs_sat['date']<pd.to_datetime('2023-12-31'))].reset_index(drop=True)

In [13]:
arroz_obs_sat.shape

(509843, 9)

In [14]:
arroz_obs_sat.to_parquet('data/arroz_obs_sat.parquet')